In [1]:
import os
import re
import tqdm
import string
import pandas as pd
import numpy as np
import util
from sklearn.decomposition import TruncatedSVD
#from data_preprocessing import *
import word_embedding_load as wel

In [2]:
[all_data, train_size, test_size, train_x, train_y, test_x] = util.loadData()

  ID    Gene             Variation  \
0  0  FAM58A  Truncating Mutations   
1  1     CBL                 W802*   
2  2     CBL                 Q249E   
3  3     CBL                 N454D   
4  4     CBL                 L399V   

                                                Text  
0  Cyclin-dependent kinases (CDKs) regulate a var...  
1   Abstract Background  Non-small cell lung canc...  
2   Abstract Background  Non-small cell lung canc...  
3  Recent evidence has demonstrated that acquired...  
4  Oncogenic mutations in the monomeric Casitas B...  


In [3]:
sentences = util.data_preprocess(all_data)

In [4]:
print(sentences[0])

LabeledSentence(['cyclindependent', 'kinases', 'cdks', 'regulate', 'variety', 'fundamental', 'cellular', 'processes', 'cdk10', 'stands', 'one', 'last', 'orphan', 'cdks', 'activating', 'cyclin', 'identified', 'kinase', 'activity', 'revealed', 'previous', 'work', 'shown', 'cdk10', 'silencing', 'increases', 'ets2', 'vets', 'erythroblastosis', 'virus', 'e26', 'oncogene', 'homolog', '2', 'driven', 'activation', 'mapk', 'pathway', 'confers', 'tamoxifen', 'resistance', 'breast', 'cancer', 'cells', 'precise', 'mechanisms', 'cdk10', 'modulates', 'ets2', 'activity', 'generally', 'functions', 'cdk10', 'remain', 'elusive', 'demonstrate', 'cdk10', 'cyclindependent', 'kinase', 'identifying', 'cyclin', 'activating', 'cyclin', 'cyclin', 'm', 'orphan', 'cyclin', 'product', 'fam58a', 'whose', 'mutations', 'cause', 'star', 'syndrome', 'human', 'developmental', 'anomaly', 'whose', 'features', 'include', 'toe', 'syndactyly', 'telecanthus', 'anogenital', 'renal', 'malformations', 'show', 'star', 'syndromeas

In [168]:
Text_INPUT_DIM=200
param = util.Doc2VecParam(1, 5, 200, 1e-4, 5, 4, 30, 1)
filename='../model/doc2vec/docEmbeddings_30_load_all.d2v'


In [169]:
GENE_INPUT_DIM=25
TEXT_INPUT_DIM=200
svd = TruncatedSVD(n_components=GENE_INPUT_DIM, random_state=12)

In [8]:
text_model = wel.getTextModel(sentences, param, filename)
truncated_one_hot_gene = wel.getGeneVec(all_data, svd)
truncated_one_hot_variation = wel.getVariationVec(all_data, svd)
text_train_arrays, text_test_arrays = wel.getTextVec(text_model, train_size, test_size, TEXT_INPUT_DIM)

Loading model...
successfully loaded the textmodel from ../model/doc2vec/docEmbeddings_30_load_all.d2v


In [9]:
print(text_train_arrays.shape)
print(text_test_arrays.shape)
text_train_arrays[0]

(3689, 200)
(986, 200)


array([-4.32804871, -1.51728463, -1.11980414, -1.48884988, -1.94485927,
        2.92375898,  0.50411427, -1.12256765,  3.63756609,  0.98047012,
       -3.59505153, -1.69239998,  1.08154798,  0.04358532, -1.45718324,
       -1.40607941, -0.81513387, -1.83771491,  0.84762865,  1.21229076,
        2.47014523, -2.76897311,  1.76767576,  0.63634545, -1.18918824,
        0.41207117,  0.46613771, -0.66748267, -0.08578082,  0.74831831,
       -0.27361712, -1.59590268, -0.68817914,  1.80353093, -2.18088555,
       -0.95929134,  3.10893011,  1.76786911, -0.6726135 ,  1.6783632 ,
        0.86585337,  2.32940316,  3.42221165, -1.29582059,  1.10946131,
       -1.03413177,  1.08570337, -1.0144273 ,  0.31510681,  0.28210998,
        2.33720469,  0.54392654, -2.83733749, -1.03502595, -0.16337521,
       -3.04194999, -0.25847396, -2.89910054,  1.95583844,  2.05173278,
       -0.68942761,  2.17256355, -2.50274634,  2.23825169, -2.29581237,
        4.42594433,  2.62937164,  2.88882828, -0.28914687, -2.05

In [10]:
train_set = np.hstack((truncated_one_hot_gene[:train_size], truncated_one_hot_variation[:train_size], text_train_arrays))
test_set = np.hstack((truncated_one_hot_gene[train_size:], truncated_one_hot_variation[train_size:], text_test_arrays))
encoded_y = pd.get_dummies(train_y)
encoded_y = np.array(encoded_y)

In [11]:
print(train_set.shape)
print(test_set.shape)

(3689, 250)
(986, 250)


In [12]:
print(train_set[0, :25])
print(train_set[0, 25:50])
print(train_set[0, 50:])

[  2.48455806e-23   5.95345506e-19   2.55148417e-20  -3.12596635e-22
   4.07695291e-22   5.14309572e-25   2.12813627e-24  -2.97221364e-27
   2.40533195e-28  -2.34699855e-30  -1.02435863e-29  -2.70616608e-28
   3.11053711e-28   1.52301261e-27  -6.31031488e-27  -4.29210763e-27
  -2.58870185e-26   1.64361290e-26  -6.65840234e-26   1.85506791e-26
  -3.44148337e-25  -1.13345002e-25   1.15081601e-24  -1.65793325e-23
   4.35367578e-23]
[  1.00000000e+00   7.43920261e-19   1.86967768e-19   2.36987218e-29
   3.78288394e-25   7.97142704e-23  -1.06895319e-20   6.87926690e-21
   4.36854413e-21  -4.04470778e-21   8.91881464e-21   6.59040569e-18
   5.67979623e-19  -3.15512997e-18  -9.40491899e-18  -6.36407932e-18
  -3.20292165e-18   9.53349364e-18   7.28293876e-18   2.84124514e-17
  -1.85442723e-17   1.28733104e-17   1.05102497e-17   4.67448189e-17
  -8.77125342e-21]
[-4.32804871 -1.51728463 -1.11980414 -1.48884988 -1.94485927  2.92375898
  0.50411427 -1.12256765  3.63756609  0.98047012 -3.59505153 

In [13]:
print(encoded_y.shape)
print(encoded_y[300])
print(encoded_y[1000])

(3689, 9)
[0 0 0 0 0 0 1 0 0]
[0 0 0 1 0 0 0 0 0]


In [14]:
v1 = train_set[0]
v2 = train_set[1]

In [ ]:
np.dot(v1, v2)

In [ ]:
from baseline_classification import *

In [ ]:
computeSim(v2, v1, [25, 25, 200])

In [34]:
print(encoded_y.shape)

(3689, 9)


In [36]:
def getLabels(encoding):
    return [ np.where(r==1)[0][0] for r in encoding ]

In [77]:
y = np.array(getLabels(encoded_y))
X = np.array(train_set)
print(X.shape)
print(y.shape)

(3689, 250)
(3689,)


In [78]:
print(y[0:10])
print(X[0:10])

[0 1 1 2 3 3 4 0 3 3]
[[  2.48455806e-23   5.95345506e-19   2.55148417e-20 ...,   7.13557661e-01
   -1.90010107e+00  -5.39088547e-01]
 [ -8.97003400e-21  -2.56787169e-16  -1.19273061e-15 ...,   2.48599529e+00
    1.99950952e-02   1.58506441e+00]
 [ -1.01263684e-20  -2.87102850e-16  -2.91398299e-16 ...,   2.54142594e+00
   -4.61241193e-02   1.78198898e+00]
 ..., 
 [ -1.01210515e-20  -2.86650396e-16  -3.14243845e-16 ...,   2.78275681e+00
   -7.79538870e-01   2.13473153e+00]
 [ -1.01210509e-20  -2.86650392e-16  -3.14244014e-16 ...,   3.60708332e+00
    9.91484880e-01   2.21232915e+00]
 [ -1.01210509e-20  -2.86650392e-16  -3.14244015e-16 ...,   4.12722254e+00
    6.26037717e-01   2.80597425e+00]]


In [166]:
def myKNN(X, y, skf):
    # metrics support: ACC, NMI, Confusion_matrix, 

    #skf = StratifiedKFold(n_splits=10, random_state = 66, shuffle = True)
    ret = []
    mat = 0
    LOSS = []
    ACC = []
    NMI = []

    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        neigh = KNeighborsClassifier(n_neighbors=3)
        neigh.fit(X_train, y_train)
        #clf = NearestCentroid()
        #clf.fit(X_train, y_train)
        y_pred = neigh.predict(X_test)
        y_pred_proba = neigh.predict_proba(X_test)
        acc = accuracy_score(y_test, y_pred)
        nmi = normalized_mutual_info_score(y_pred, y_test)
        loss = log_loss(y_test, y_pred_proba)
        ACC.append(acc)
        LOSS.append(loss)
        NMI.append(nmi)

        #print("acc: " + str(acc))
        #print("NMI: " + str(nmi))
        #print("log_loss: " + str(log_loss))
        cnf_matrix = confusion_matrix(y_test, y_pred)
        mat = mat + cnf_matrix

    mat = mat / 10
    print("Accuracy: %.4f ± %.4f" % (np.mean(ACC), np.std(ACC)))
    print("NMI: %.4f ± %.4f" % (np.mean(NMI), np.std(NMI)))
    print("Log_loss: %.4f ± %.4f" % (np.mean(LOSS), np.std(LOSS)))
    print(mat)

In [167]:
ret = myKNN(X, y, StratifiedKFold(n_splits=10, random_state = 66, shuffle = True))

Accuracy: 0.6110 ± 0.0214
NMI: 0.3925 ± 0.0274
Log_loss: 6.5794 ± 0.5453
[[ 40.2   1.8   0.7  15.3   4.4   2.1   1.6   0.1   0. ]
 [  3.5  24.8   0.6   1.7   0.9   1.   17.3   0.    0. ]
 [  0.9   0.3   3.7   1.9   0.4   0.    2.4   0.    0. ]
 [ 17.4   1.6   0.8  50.6   2.4   0.4   1.8   0.    0.1]
 [  7.1   1.5   0.8   3.4   9.6   1.1   3.2   0.    0. ]
 [  4.3   3.    0.2   1.5   1.   17.5   2.1   0.1   0. ]
 [  5.2  16.9   3.4   1.9   1.7   0.7  75.2   0.    0.4]
 [  0.3   0.5   0.    0.1   0.    0.    0.3   0.4   0.5]
 [  0.4   0.1   0.    0.1   0.1   0.    0.2   0.    3.4]]


In [105]:
ret[0].shape

(374,)